In [ ]:
# append the parent directory to the module search path. this is required to run the notebook from this folder
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pickle

In [ ]:
# avoid type-3 fonts
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

sns.set_style("white")
sns.set_context("notebook", rc={'axes.linewidth': 2, 'grid.linewidth': 1},  font_scale=2.5)

## CIFAR-10 gradient norm regularization

In [ ]:
# load the results from the experiment 
path = '../experiments/results/cifar10_gnorm_lpips_conditional'
results = {}
for file in os.listdir(path):
    if file[-4:] != '.pkl':
        continue
    pos = file.find('gnorm_const=') 
    pos = pos + len('gnorm_const=')
    reg_const = file[pos:pos+file[pos:].find('_')]
    reg_const = float(reg_const)

    distances = pickle.load(open(os.path.join(path, file), 'rb'))
    if not reg_const in results:
        results[reg_const] = []
    results[reg_const].append(distances)

In [ ]:
accuracies = pickle.load(open('gnorm_accuracies.pkl', 'rb'))

In [ ]:
gnorm = list(results.keys())
gnorm.sort()

In [ ]:

plt.figure(figsize=(10,6))
ax1 = plt.gca()
ax2 = ax1.twinx()

y = [np.mean(accuracies[g])/100 for g in gnorm]
y1 = [np.min(accuracies[g])/100 for g in gnorm]
y2 = [np.max(accuracies[g])/100 for g in gnorm]

ax1.plot(gnorm, y, 'o--', ms=8, color='#1f77b4', label='Accuracy')
ax1.fill_between(gnorm, y1, y2, color='#1f77b4', alpha=.25)
ax1.set_ylim([0.1,1.15])
ax1.set_ylabel('Accuracy', color='#1f77b4', weight='bold')
ax1.yaxis.label.set_color('#1f77b4')
ax1.tick_params(axis='y', colors='#1f77b4')


mean_values = {k:np.mean([np.mean(l) for l in v])  for k, v in results.items()}
y = [1-mean_values[v] for v in gnorm]
min_values = {k:np.min([np.mean(l) for l in v])  for k, v in results.items()}
y1 = [1-min_values[v] for v in gnorm]
max_values = {k:np.max([np.mean(l) for l in v])  for k, v in results.items()}
y2 = [1-max_values[v] for v in gnorm]

ax2.plot(gnorm, y, 'o--', ms=8, color='#ff7f0e')
ax2.fill_between(gnorm, y1, y2, color='#ff7f0e', alpha=.25)
ax2.set_ylim([0.7, 0.89])
#ax2.set_ylabel('1-LPIPS', color='#ff7f0e')
ax2.set_yticks([])

ax1.set_xlabel('Regularization Constant')
plt.xscale('log')
#plt.title('Gradient Norm (CIFAR-10)')
#ax1.legend(loc='lower center')
plt.savefig('../figures/cifar10_gnorm_lpips.png', bbox_inches='tight')

plt.show()

In [ ]:
# load the results from the experiment 
path = '../experiments/results/manifold_robustness'
results = {}
for file in os.listdir(path):
    if file[-4:] != '.pkl':
        continue
    pos = file.find('gnorm_const=') 
    pos = pos + len('gnorm_const=')
    reg_const = file[pos:pos+file[pos:].find('_')]
    reg_const = float(reg_const)

    on_robustness, off_robustnes  = pickle.load(open(os.path.join(path, file), 'rb'))
    if not reg_const in results:
        results[reg_const] = []
    results[reg_const].append(({k:np.mean(v) for k, v in on_robustness.items()}, {k:np.mean(v) for k, v in off_robustnes.items()}))

In [ ]:
step_sizes = np.logspace(-3, 2, num=20)
step_sizes = np.insert(step_sizes, 0, 0)

In [ ]:
plt.figure(figsize=(10,6))
step = 14

plot_values = []
for gn in gnorm:
    replication_values = []
    for on_robustness, _ in results[gn]: # for each replication
        replication_values.append(on_robustness[step_sizes[step]])
    plot_values.append(np.mean(replication_values))
plt.plot(gnorm, plot_values, 'o--', label='On-Manifold Perturbation', ms=8, color='#2ca02c')

y1 = []
for gn in gnorm:
    replication_values = []
    for on_robustness, _ in results[gn]: # for each replication
        replication_values.append(on_robustness[step_sizes[step]])
    y1.append(np.min(replication_values))
y2 = []
for gn in gnorm:
    replication_values = []
    for on_robustness, _ in results[gn]: # for each replication
        replication_values.append(on_robustness[step_sizes[step]])
    y2.append(np.max(replication_values))

plt.fill_between(gnorm, y1, y2, color='#2ca02c', alpha=.25)


plot_values = []
for gn in gnorm:
    replication_values = []
    for _, off_robustness in results[gn]: # for each replication
        replication_values.append(off_robustness[step_sizes[step]])
    plot_values.append(np.mean(replication_values))
plt.plot(gnorm, plot_values, 'o--', label='Off-Manifold Perturbation', color='red', ms=8)

y1 = []
for gn in gnorm:
    replication_values = []
    for _, off_robustness in results[gn]: # for each replication
        replication_values.append(off_robustness[step_sizes[step]])
    y1.append(np.min(replication_values))
y2 = []
for gn in gnorm:
    replication_values = []
    for _, off_robustness in results[gn]: # for each replication
        replication_values.append(off_robustness[step_sizes[step]])
    y2.append(np.max(replication_values))

plt.fill_between(gnorm, y1, y2, color='#d62728', alpha=.25)

plt.xscale('log')
#plt.xlabel('Regularization Constant')
plt.gca().set_xticks([])
plt.ylabel('L1-Norm', weight='bold')
plt.title(f'Gradient Norm (CIFAR-10)')
#plt.legend(fontsize=20)
plt.ylim([0, 1.9])
plt.savefig('../figures/cifar10_gnorm_perturbation.png', bbox_inches='tight')
plt.show()


## CIFAR-10 smoothness regularization

In [ ]:
# load the results from the experiment 
path = '../experiments/results/cifar10_smooth_lpips_conditional'
results = {}
for file in os.listdir(path):
    if file[-4:] != '.pkl':
        continue
    pos = file.find('gnorm_const=') 
    pos = pos + len('gnorm_const=')
    reg_const = file[pos:pos+file[pos:].find('_')]
    reg_const = float(reg_const)

    distances = pickle.load(open(os.path.join(path, file), 'rb'))
    if not reg_const in results:
        results[reg_const] = []
    results[reg_const].append(distances)

In [ ]:
accuracies = pickle.load(open('smooth_accuracies.pkl', 'rb'))

In [ ]:
gnorm = list(results.keys())
gnorm.sort()

In [ ]:
plt.figure(figsize=(10,6))
ax1 = plt.gca()
ax2 = ax1.twinx()

y = [np.mean(accuracies[g])/100 for g in gnorm]
y1 = [np.min(accuracies[g])/100 for g in gnorm]
y2 = [np.max(accuracies[g])/100 for g in gnorm]

ax1.plot(gnorm, y, 'o--', ms=8, color='#1f77b4', label='Accuracy')
ax1.fill_between(gnorm, y1, y2, color='#1f77b4', alpha=.25)
ax1.set_ylim([0.1,1.15])
#ax1.set_ylabel('Accuracy', color='#1f77b4')
ax1.set_yticks([])

mean_values = {k:np.mean([np.mean(l) for l in v])  for k, v in results.items()}
y = [1-mean_values[v] for v in gnorm]
min_values = {k:np.min([np.mean(l) for l in v])  for k, v in results.items()}
y1 = [1-min_values[v] for v in gnorm]
max_values = {k:np.max([np.mean(l) for l in v])  for k, v in results.items()}
y2 = [1-max_values[v] for v in gnorm]

ax2.plot(gnorm, y, 'o--', ms=8, color='#ff7f0e')
ax2.fill_between(gnorm, y1, y2, color='#ff7f0e', alpha=.25)
ax2.set_ylim([0.65, 0.89])
#ax2.set_ylabel('1-LPIPS', color='#ff7f0e')
ax2.tick_params(axis='y', colors='#ff7f0e')


ax1.set_xlabel('Regularization Constant')
plt.xscale('log')
#plt.title('Smoothness Penalty (CIFAR-10)')
plt.savefig('../figures/cifar10_smooth_lpips.png', bbox_inches='tight')

#ax2.legend()
plt.show()

In [ ]:
path = '../experiments/results/cifar10_smooth_manifold_robustness'
results = {}
for file in os.listdir(path):
    if file[-4:] != '.pkl':
        continue
    pos = file.find('gnorm_const=') 
    pos = pos + len('gnorm_const=')
    reg_const = file[pos:pos+file[pos:].find('_')]
    reg_const = float(reg_const)

    on_robustness, off_robustnes  = pickle.load(open(os.path.join(path, file), 'rb'))
    if not reg_const in results:
        results[reg_const] = []
    results[reg_const].append(({k:np.mean(v) for k, v in on_robustness.items()}, {k:np.mean(v) for k, v in off_robustnes.items()}))

In [ ]:
step_sizes = np.logspace(-3, 2, num=20)
step_sizes = np.insert(step_sizes, 0, 0)

In [ ]:
plt.figure(figsize=(10,6))
step = 14

plot_values = []
for gn in gnorm:
    replication_values = []
    for on_robustness, _ in results[gn]: # for each replication
        replication_values.append(on_robustness[step_sizes[step]])
    plot_values.append(np.mean(replication_values))
plt.plot(gnorm, plot_values, 'o--', label='On-Manifold Perturbation', ms=8, color='#2ca02c')

y1 = []
for gn in gnorm:
    replication_values = []
    for on_robustness, _ in results[gn]: # for each replication
        replication_values.append(on_robustness[step_sizes[step]])
    y1.append(np.min(replication_values))
y2 = []
for gn in gnorm:
    replication_values = []
    for on_robustness, _ in results[gn]: # for each replication
        replication_values.append(on_robustness[step_sizes[step]])
    y2.append(np.max(replication_values))

plt.fill_between(gnorm, y1, y2, color='#2ca02c', alpha=.25)


plot_values = []
for gn in gnorm:
    replication_values = []
    for _, off_robustness in results[gn]: # for each replication
        replication_values.append(off_robustness[step_sizes[step]])
    plot_values.append(np.mean(replication_values))
plt.plot(gnorm, plot_values, 'o--', label='Off-Manifold Perturbation', color='#d62728', ms=8)

y1 = []
for gn in gnorm:
    replication_values = []
    for _, off_robustness in results[gn]: # for each replication
        replication_values.append(off_robustness[step_sizes[step]])
    y1.append(np.min(replication_values))
y2 = []
for gn in gnorm:
    replication_values = []
    for _, off_robustness in results[gn]: # for each replication
        replication_values.append(off_robustness[step_sizes[step]])
    y2.append(np.max(replication_values))

plt.fill_between(gnorm, y1, y2, color='#d62728', alpha=.25)

plt.xscale('log')
#plt.xlabel('Regularization Constant')
plt.gca().set_xticks([])
#plt.ylabel('L1 Softmax Deviation')
plt.title(f'Smoothness Penalty (CIFAR-10)')
plt.ylim([0, 1.9])
plt.yticks([])
plt.legend(fontsize=25)
plt.savefig('../figures/cifar10_smooth_perturbation.png', bbox_inches='tight')
plt.show()


## CIFAR-10 Randomized Smoothing

In [ ]:
# load the results from the experiment 
path = '../experiments/results/cifar10_randomized_smoothing_lpips_conditional'
results = {}
for file in os.listdir(path):
    if file[-4:] != '.pkl':
        continue
    pos = file.find('rand_smooth_noise_level=') 
    pos = pos + len('rand_smooth_noise_level=')
    reg_const = file[pos:pos+file[pos:].find('_')]
    reg_const = float(reg_const)

    distances = pickle.load(open(os.path.join(path, file), 'rb'))
    if not reg_const in results:
        results[reg_const] = []
    results[reg_const].append(distances)

In [ ]:
accuracies = pickle.load(open('randomized_smoothing_accuracies.pkl', 'rb'))

In [ ]:
gnorm = list(results.keys())
gnorm.sort()
gnorm = gnorm[0:27]

In [ ]:
plt.figure(figsize=(10,6))
ax1 = plt.gca()
ax2 = ax1.twinx()

y = [np.mean(accuracies[g])/100 for g in gnorm]
y1 = [np.min(accuracies[g])/100 for g in gnorm]
y2 = [np.max(accuracies[g])/100 for g in gnorm]

ax1.plot(gnorm, y, 'o--', ms=8, color='#1f77b4', label='Accuracy')
ax1.fill_between(gnorm, y1, y2, color='#1f77b4', alpha=.25)
ax1.set_ylim([0.1,1.15])
#ax1.set_ylabel('Accuracy', color='#1f77b4')
ax1.set_yticks([])

mean_values = {k:np.mean([np.mean(l) for l in v])  for k, v in results.items()}
y = [1-mean_values[v] for v in gnorm]
min_values = {k:np.min([np.mean(l) for l in v])  for k, v in results.items()}
y1 = [1-min_values[v] for v in gnorm]
max_values = {k:np.max([np.mean(l) for l in v])  for k, v in results.items()}
y2 = [1-max_values[v] for v in gnorm]

ax2.plot(gnorm, y, 'o--', ms=8, color='#ff7f0e')
ax2.fill_between(gnorm, y1, y2, color='#ff7f0e', alpha=.25)
ax2.set_ylim([0.2, 0.9])
ax2.set_ylabel('1-LPIPS', color='#ff7f0e', weight='bold')
ax2.tick_params(axis='y', colors='#ff7f0e')

ax1.set_xlabel('Noise Level')
plt.xscale('log')
#plt.title('Randomized Smoothing (CIFAR-10)')
plt.savefig('../figures/cifar10_randomized_smoothing_lpips.png', bbox_inches='tight')

plt.show()

In [ ]:
path = '../experiments/results/cifar10_randomized_smoothing_manifold_robustness'
results = {}
for file in os.listdir(path):
    if file[-4:] != '.pkl':
        continue
    pos = file.find('rand_smooth_noise_level=') 
    pos = pos + len('rand_smooth_noise_level=')
    reg_const = file[pos:pos+file[pos:].find('_')]
    reg_const = float(reg_const)

    on_robustness, off_robustnes  = pickle.load(open(os.path.join(path, file), 'rb'))
    if not reg_const in results:
        results[reg_const] = []
    results[reg_const].append(({k:np.mean(v) for k, v in on_robustness.items()}, {k:np.mean(v) for k, v in off_robustnes.items()}))

In [ ]:
step_sizes = np.logspace(-3, 2, num=20)
step_sizes = np.insert(step_sizes, 0, 0)

In [ ]:
plt.figure(figsize=(10,6))
step = 14 # the size of the perturbation that we consider

plot_values = []
for gn in gnorm:
    replication_values = []
    for on_robustness, _ in results[gn]: # for each replication
        replication_values.append(on_robustness[step_sizes[step]])
    plot_values.append(np.mean(replication_values))
plt.plot(gnorm, plot_values, 'o--', label='On-Manifold Perturbation', ms=8, color='#2ca02c')

y1 = []
for gn in gnorm:
    replication_values = []
    for on_robustness, _ in results[gn]: # for each replication
        replication_values.append(on_robustness[step_sizes[step]])
    y1.append(np.min(replication_values))
y2 = []
for gn in gnorm:
    replication_values = []
    for on_robustness, _ in results[gn]: # for each replication
        replication_values.append(on_robustness[step_sizes[step]])
    y2.append(np.max(replication_values))

plt.fill_between(gnorm, y1, y2, color='#2ca02c', alpha=.25)

plot_values = []
for gn in gnorm:
    replication_values = []
    for _, off_robustness in results[gn]: # for each replication
        replication_values.append(off_robustness[step_sizes[step]])
    plot_values.append(np.mean(replication_values))
plt.plot(gnorm, plot_values, 'o--', label='Off-Manifold Perturbation', color='#d62728', ms=8)

y1 = []
for gn in gnorm:
    replication_values = []
    for _, off_robustness in results[gn]: # for each replication
        replication_values.append(off_robustness[step_sizes[step]])
    y1.append(np.min(replication_values))
y2 = []
for gn in gnorm:
    replication_values = []
    for _, off_robustness in results[gn]: # for each replication
        replication_values.append(off_robustness[step_sizes[step]])
    y2.append(np.max(replication_values))

plt.fill_between(gnorm, y1, y2, color='#d62728', alpha=.25)

#plt.ylabel('L1 Softmax Deviation')
#plt.xlabel('Noise Level')

plt.xscale('log')
plt.title('Randomized Smoothing (CIFAR-10)')
#plt.legend(fontsize=20)
plt.ylim([0, 1.9])
plt.yticks([])
plt.xticks([])
plt.savefig('../figures/cifar10_randomized_smoothing_perturbation.png', bbox_inches='tight')
plt.show()


## CIFAR-10 Adversarial Robust

In [ ]:
# load the results from the experiment 
path = '../experiments/results/cifar10_adversarial_score_alignment_lpips'
results = {}
for file in os.listdir(path):
    if file[-4:] != '.pkl':
        continue
    pos = file.find('epsilon=') 
    pos = pos + len('epsilon=')
    reg_const = file[pos:pos+file[pos:].find('_')]
    reg_const = float(reg_const)

    distances = pickle.load(open(os.path.join(path, file), 'rb'))
    if not reg_const in results:
        results[reg_const] = []
    results[reg_const].append(distances)

In [ ]:
accuracies = pickle.load(open('adversarial_accuracies.pkl', 'rb'))

In [ ]:
del accuracies[21]

In [ ]:
gnorm = list(results.keys())
gnorm.sort()

In [ ]:
plt.figure(figsize=(10,6))
ax1 = plt.gca()
ax2 = ax1.twinx()

y = np.array(accuracies) / 100
y1 = np.array(accuracies) / 100
y2 = np.array(accuracies) / 100

ax1.plot(gnorm, y, 'o--', ms=8, color='#1f77b4', label='Accuracy')
ax1.fill_between(gnorm, y1, y2, color='#1f77b4', alpha=.25)
ax1.set_ylim([0.05,1.15])
ax1.set_ylabel('Accuracy', color='#1f77b4', weight='bold')
ax1.yaxis.label.set_color('#1f77b4')
ax1.tick_params(axis='y', colors='#1f77b4')

mean_values = {k:np.mean([np.mean(l) for l in v])  for k, v in results.items()}
y = [1-mean_values[v] for v in gnorm]
min_values = {k:np.min([np.mean(l) for l in v])  for k, v in results.items()}
y1 = [1-min_values[v] for v in gnorm]
max_values = {k:np.max([np.mean(l) for l in v])  for k, v in results.items()}
y2 = [1-max_values[v] for v in gnorm]

ax2.plot(gnorm, y, 'o--', ms=8, color='#ff7f0e')
ax2.fill_between(gnorm, y1, y2, color='#ff7f0e', alpha=.25)
ax2.set_ylim([0.65, 0.89])
ax2.set_ylabel('1-LPIPS', color='#ff7f0e', weight='bold')
ax2.tick_params(axis='y', colors='#ff7f0e')

ax1.set_xlabel('Adversarial Perturbation Budget (Epsilon)', fontsize=24)
plt.xscale('log')
plt.title('Projected Gradient Descent (CIFAR-10)', fontsize=26)
plt.savefig('../figures/cifar10_adversarial_lpips.png', bbox_inches='tight')

plt.show()

In [ ]:
path = '../experiments/results/cifar10_adversarial_manifold_robustness'
results = {}
for file in os.listdir(path):
    if file[-4:] != '.pkl':
        continue
    pos = file.find('epsilon=') 
    pos = pos + len('epsilon=')
    reg_const = file[pos:pos+file[pos:].find('_')]
    reg_const = float(reg_const)

    on_robustness, off_robustnes  = pickle.load(open(os.path.join(path, file), 'rb'))
    if not reg_const in results:
        results[reg_const] = []
    results[reg_const].append(({k:np.mean(v) for k, v in on_robustness.items()}, {k:np.mean(v) for k, v in off_robustnes.items()}))

In [ ]:
step_sizes = np.logspace(-3, 2, num=20)
step_sizes = np.insert(step_sizes, 0, 0)

In [ ]:
plt.figure(figsize=(10,6))
step = 14 # the size of the perturbation that we consider

plot_values = []
for gn in gnorm:
    replication_values = []
    for on_robustness, _ in results[gn]: # for each replication
        replication_values.append(on_robustness[step_sizes[step]])
    plot_values.append(np.mean(replication_values))
plt.plot(gnorm, plot_values, 'o--', label='On-Manifold Perturbation', ms=8, color='#2ca02c')

y1 = []
for gn in gnorm:
    replication_values = []
    for on_robustness, _ in results[gn]: # for each replication
        replication_values.append(on_robustness[step_sizes[step]])
    y1.append(np.min(replication_values))
y2 = []
for gn in gnorm:
    replication_values = []
    for on_robustness, _ in results[gn]: # for each replication
        replication_values.append(on_robustness[step_sizes[step]])
    y2.append(np.max(replication_values))

plt.fill_between(gnorm, y1, y2, color='#2ca02c', alpha=.25)

plot_values = []
for gn in gnorm:
    replication_values = []
    for _, off_robustness in results[gn]: # for each replication
        replication_values.append(off_robustness[step_sizes[step]])
    plot_values.append(np.mean(replication_values))
plt.plot(gnorm, plot_values, 'o--', label='Off-Manifold Perturbation', color='#d62728', ms=8)

y1 = []
for gn in gnorm:
    replication_values = []
    for _, off_robustness in results[gn]: # for each replication
        replication_values.append(off_robustness[step_sizes[step]])
    y1.append(np.min(replication_values))
y2 = []
for gn in gnorm:
    replication_values = []
    for _, off_robustness in results[gn]: # for each replication
        replication_values.append(off_robustness[step_sizes[step]])
    y2.append(np.max(replication_values))

plt.fill_between(gnorm, y1, y2, color='#d62728', alpha=.25)

plt.ylabel('L1-Norm', weight='bold')

plt.xscale('log')
plt.title('Projected Gradient Descent (CIFAR-10)', fontsize=26)
plt.xlabel('Adversarial Perturbation Budget (Epsilon)', fontsize=24)
plt.legend(fontsize=20)
plt.ylim([0, 1.9])
plt.savefig('../figures/cifar10_adversarial_perturbation.png', bbox_inches='tight')
plt.show()